# Deciphering Code with Character-Level RNN





## Dataset

In [1]:
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [2]:
df = pd.read_csv('../data/Vigenere.csv')
df

,Passwords,ciphertext,key
0,2428031609,yux6njoswz,6058xqxw
1,4531040045,x4s5cmclx4,39zemsmv
2,almigrana1,jjfflk9ijz,j837f395
3,quiero95,2h5e32dn,mxxamoes
4,doitnow2,luibyzc7,igasllqf
...,...,...,...
199995,dime168,es02coa,bkoylscm
199996,!l0v3k3v!n,!90iecvx!q,yaxl22cd
199997,9870tmf05,7pf8j4ad3,8rii0s5n
199998,CAMILA,wqj7jx,uq7z8xdh


In [3]:
for i in range(len(df['Passwords'])):
    if len(df['Passwords'][i]) > 20:
        df.drop(i, axis=0, inplace=True)

In [4]:
df = df.dropna()[:120000].reset_index(drop=True)

In [5]:
df['Passwords'].apply(str)
df['ciphertext'].apply(str)
df

,Passwords,ciphertext,key
0,2428031609,yux6njoswz,6058xqxw
1,4531040045,x4s5cmclx4,39zemsmv
2,almigrana1,jjfflk9ijz,j837f395
3,quiero95,2h5e32dn,mxxamoes
4,doitnow2,luibyzc7,igasllqf
...,...,...,...
119995,6942498317,qp1r2wbaln,uq7z8xdh
119996,babycake00,ckpmnsmq1a,bkoylscm
119997,m91485,ywo4kj,mxxamoes
119998,5215hooker,e0u2mhnfnp,j837f395


## Preprocessing Data

In [6]:
def tokenize(x):
    x_tk = Tokenizer(char_level=True)
    x_tk.fit_on_texts(x)                 
    return x_tk.texts_to_sequences(x), x_tk

def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen=length, padding="post", truncating="post",)

### Preprocess Pipeline

In [7]:
def preprocess(x, y):
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

In [8]:
preproc_code_sentences, preproc_plaintext_sentences, code_tokenizer, plaintext_tokenizer = preprocess(df['Passwords'], df['ciphertext'])

In [9]:
preproc_code_sentences[0]

array([ 7, 17,  7, 15,  5, 13,  3, 19,  5, 12,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0])

In [10]:
len(code_tokenizer.word_index)+1

126

In [11]:
len(plaintext_tokenizer.word_index)+1

126

In [12]:
plaintext_tokenizer.word_index

{'k': 1,
 'o': 2,
 'g': 3,
 'a': 4,
 'e': 5,
 'l': 6,
 'u': 7,
 'y': 8,
 '5': 9,
 '9': 10,
 'h': 11,
 '8': 12,
 'b': 13,
 'n': 14,
 'i': 15,
 '3': 16,
 'r': 17,
 '1': 18,
 '4': 19,
 'd': 20,
 'm': 21,
 'j': 22,
 'c': 23,
 'z': 24,
 '0': 25,
 'f': 26,
 '6': 27,
 'x': 28,
 'q': 29,
 'p': 30,
 's': 31,
 '2': 32,
 't': 33,
 '7': 34,
 'v': 35,
 'w': 36,
 '.': 37,
 '_': 38,
 '!': 39,
 '-': 40,
 '*': 41,
 '@': 42,
 ' ': 43,
 '#': 44,
 '/': 45,
 '$': 46,
 ',': 47,
 '+': 48,
 '&': 49,
 ')': 50,
 '\\': 51,
 '(': 52,
 '=': 53,
 '?': 54,
 ';': 55,
 "'": 56,
 ']': 57,
 'à': 58,
 '<': 59,
 '¸': 60,
 '%': 61,
 '~': 62,
 '[': 63,
 'ã': 64,
 '`': 65,
 ':': 66,
 '^': 67,
 '>': 68,
 '±': 69,
 '"': 70,
 '|': 71,
 '×': 72,
 '{': 73,
 '¼': 74,
 'â': 75,
 '£': 76,
 '§': 77,
 'ä': 78,
 '\x99': 79,
 '\x9f': 80,
 '³': 81,
 '¹': 82,
 '\xa0': 83,
 '°': 84,
 '\x97': 85,
 '}': 86,
 '©': 87,
 '¤': 88,
 '·': 89,
 '\x81': 90,
 'î': 91,
 '\x84': 92,
 '¶': 93,
 'ï': 94,
 '\x88': 95,
 '²': 96,
 '\x96': 97,
 '¥': 98,
 '¡'

In [13]:
preproc_code_sentences.shape

(120000, 20)

In [14]:
preproc_plaintext_sentences.shape

(120000, 20, 1)

# LSTM

In [15]:
from keras.layers import GRU, Input, Dense, TimeDistributed, LSTM
from keras.models import Model, Sequential
from keras.layers import Activation
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
import tensorflow


def lstm(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    x = Input(shape=input_shape[1:])   
    seq = LSTM(units= 128, return_sequences = True, name='Layer1')(x)
    seq = LSTM(units= 128, return_sequences = True, name='Layer2')(seq)
    output = TimeDistributed(Dense(units = plaintext_vocab_size, activation='softmax', name='Layer3'))(seq)
    model = Model(inputs = x, outputs = output)
    model.compile(optimizer='adam', loss=sparse_categorical_crossentropy, metrics=['accuracy'])
    model.summary()
    return model

tmp_x = pad(preproc_code_sentences, preproc_plaintext_sentences.shape[1]) 
tmp_x = tmp_x.reshape((-1, preproc_plaintext_sentences.shape[-2], 1))     

In [16]:
tmp_x.shape

(120000, 20, 1)

In [17]:
lstm_model = lstm(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20, 1)]           0         
_________________________________________________________________
Layer1 (LSTM)                (None, 20, 128)           66560     
_________________________________________________________________
Layer2 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
time_distributed (TimeDistri (None, 20, 126)           16254     
Total params: 214,398
Trainable params: 214,398
Non-trainable params: 0
_________________________________________________________________


In [18]:
lstm_model.fit(tmp_x, preproc_plaintext_sentences, batch_size=512, epochs=15, validation_split=0.3)

Epoch 1/15
165/165 [==============================] - 22s 24ms/step - loss: 2.2944 - accuracy: 0.5489 - val_loss: 1.6182 - val_accuracy: 0.5762
Epoch 2/15
165/165 [==============================] - 3s 18ms/step - loss: 1.6039 - accuracy: 0.5776 - val_loss: 1.5951 - val_accuracy: 0.5783
Epoch 3/15
165/165 [==============================] - 3s 18ms/step - loss: 1.5886 - accuracy: 0.5782 - val_loss: 1.5842 - val_accuracy: 0.5780
Epoch 4/15
165/165 [==============================] - 3s 18ms/step - loss: 1.5771 - accuracy: 0.5791 - val_loss: 1.5749 - val_accuracy: 0.5787
Epoch 5/15
165/165 [==============================] - 3s 18ms/step - loss: 1.5669 - accuracy: 0.5812 - val_loss: 1.5661 - val_accuracy: 0.5799
Epoch 6/15
165/165 [==============================] - 3s 18ms/step - loss: 1.5594 - accuracy: 0.5819 - val_loss: 1.5594 - val_accuracy: 0.5815
Epoch 7/15
165/165 [==============================] - 3s 18ms/step - loss: 1.5560 - accuracy: 0.5825 - val_loss: 1.5545 - val_accuracy: 0.582

In [19]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = ''
    return ''.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

def getPred(model, x, n):
    print('Predicted\tActual')
    for i in range(n):
        print(logits_to_text(model.predict([x[:n]])[i], plaintext_tokenizer), end='\t')
        print(df['Passwords'][i])

In [20]:
getPred(lstm_model, tmp_x, 5)

Predicted	Actual
b4e9tkigik	2428031609
4nt1tkiigr	4531040045
jbt0iks5sw	almigrana1
dnb135kn	quiero95
deb9kzve	doitnow2


# GRU

In [21]:
def gru(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    x = Input(shape=input_shape[1:])   
    seq = GRU(units= 128, return_sequences = True, name='Layer1')(x)
    seq = GRU(units= 128, return_sequences = True, name='Layer2')(seq)
    output = TimeDistributed(Dense(units = plaintext_vocab_size, activation='softmax', name='Layer3'))(seq)
    model = Model(inputs = x, outputs = output)
    model.compile(optimizer='adam', loss= sparse_categorical_crossentropy, metrics=['accuracy'])
    model.summary()
    return model

In [22]:
gru_model = gru(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 20, 1)]           0         
_________________________________________________________________
Layer1 (GRU)                 (None, 20, 128)           50304     
_________________________________________________________________
Layer2 (GRU)                 (None, 20, 128)           99072     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 126)           16254     
Total params: 165,630
Trainable params: 165,630
Non-trainable params: 0
_________________________________________________________________


In [23]:
gru_model.fit(tmp_x, preproc_plaintext_sentences, batch_size=512, epochs=15, validation_split=0.3)

Epoch 1/15
165/165 [==============================] - 7s 21ms/step - loss: 2.2630 - accuracy: 0.5515 - val_loss: 1.6036 - val_accuracy: 0.5768
Epoch 2/15
165/165 [==============================] - 3s 16ms/step - loss: 1.5944 - accuracy: 0.5773 - val_loss: 1.5886 - val_accuracy: 0.5770
Epoch 3/15
165/165 [==============================] - 3s 16ms/step - loss: 1.5812 - accuracy: 0.5791 - val_loss: 1.5823 - val_accuracy: 0.5785
Epoch 4/15
165/165 [==============================] - 3s 16ms/step - loss: 1.5752 - accuracy: 0.5810 - val_loss: 1.5755 - val_accuracy: 0.5799
Epoch 5/15
165/165 [==============================] - 3s 16ms/step - loss: 1.5657 - accuracy: 0.5826 - val_loss: 1.5630 - val_accuracy: 0.5822
Epoch 6/15
165/165 [==============================] - 3s 16ms/step - loss: 1.5563 - accuracy: 0.5831 - val_loss: 1.5528 - val_accuracy: 0.5829
Epoch 7/15
165/165 [==============================] - 3s 16ms/step - loss: 1.5469 - accuracy: 0.5843 - val_loss: 1.5482 - val_accuracy: 0.5826

In [24]:
getPred(gru_model, tmp_x, 5)

Predicted	Actual
b4p9t3j909	2428031609
44tct30099	4531040045
j9t0h3s3sj	almigrana1
7hbq3g99	quiero95
4ubt3g79	doitnow2
